In [1]:
import numpy as np
import math

### Read Input


In [2]:
incom = np.loadtxt('14_input.txt',delimiter="\n", comments=None,dtype=str)

incom[:5]

array(['118 ORE => 7 GTPZ', '6 RNQJN, 4 NQKVW => 4 DTQRC',
       '2 GBXJL => 3 XHBR', '4 BPZM => 9 LVDRH', '131 ORE => 3 RHBL'],
      dtype='<U98')

#### digest input into chemical ins and outs

In [3]:
incom_split = [i.split(' => ') for i in incom]

incom_pies = [i[0].split(', ') +[i[1]] for i in incom_split]

incom_pies[:5]

[['118 ORE', '7 GTPZ'],
 ['6 RNQJN', '4 NQKVW', '4 DTQRC'],
 ['2 GBXJL', '3 XHBR'],
 ['4 BPZM', '9 LVDRH'],
 ['131 ORE', '3 RHBL']]

So, the first entries in a list are the inputs, the last entries are the chemicals generated.

Now, create a chemical class:

In [247]:
# Create a class

class Chem:
    def __init__(self,name):
        self.name = name
        self.recipies = []
        self.min_ore = [1,999999999] # [quantity, ore required]
        
        toe.append(self.name) # append self to table of elements
        
    def add_recipie(self, recipie):
        self.recipies.append(recipie)
        
    def min_ore_calc(self): # original
        ores = []
        for card in self.recipies:
            #quant_chem_gen = card[0][0]
            ore = 0
            for ingred in card[1:]:
                if ingred[1].min_ore[1] == 999999999:
                    ingred[1].min_ore_calc()
                ore += math.ceil(ingred[0]/ingred[1].min_ore[0])*ingred[1].min_ore[1]
            ores.append([card[0][0],ore])
        
        # Choose minimum ore:
        self.min_ore = ores[np.argmax([i[0]/i[1] for i in ores])]
 
       
    def min_ore_calc_remainders(self): # enhanced to account for remainder chemicals
        # modifies self, but also returns remainder products
        ores = []
        for card in self.recipies:
            #quant_chem_gen = card[0][0]
            remainder = np.empty((0,2))
            ore = 0
            for ingred in card[1:]: # 1: since idx 0 is the chem being prepared
                if ingred[1] != ORE:
                    remainder = np.append(remainder,ingred[1].min_ore_calc_remainders(), axis=0)
                    remainder = consol_remain(remainder)
                    
                    # if ingredient is in remainder
                    if ingred[1] in remainder:
                        if remainder[np.where(remainder[:,1] == ingred[1]),0] >= ingred[0]: # if the remainder is sufficient
                            remainder[np.where(remainder[:,1] == ingred[1]),0] -= ingred[0] # just adjust the amount of remainder                        
                        else: # make additional necessary ingredient
                            ore += math.ceil((ingred[0]-remainder[np.where(remainder[:,1] == ingred[1]),0])/ingred[1].min_ore[0])*ingred[1].min_ore[1]
                            remainder = np.append(remainder, [[int(ingred[1].min_ore[0]-((ingred[0]-remainder[np.where(remainder[:,1] == ingred[1]),0])%ingred[1].min_ore[0]))
                                                               ,ingred[1]]], axis=0)
                                                            
                    else:
                        ore += math.ceil(ingred[0]/ingred[1].min_ore[0])*ingred[1].min_ore[1]
                        remainder = np.append(remainder, [[int(ingred[1].min_ore[0]-(ingred[0]%ingred[1].min_ore[0])) ,ingred[1]]], axis=0)
                    
            ores.append([card[0][0],ore])
            
        
        # Choose minimum ore:
        #if self != ORE:
            #self.min_ore = ores[np.argmax([i[0]/i[1] for i in ores])]
            
        
        # Return remainder
        return remainder

In [248]:
# Define a function that consolidates multiple entries on the remainders array...

def consol_remain(remainder):
    out = np.empty((0,2))
    
    while remainder.shape[0] > 0:
        chem = remainder[0,1]
        out = np.append(out, [[np.sum([i for i in a[np.where(a[:,1] == chem),0]]),chem]], axis=0)
        remainder = np.delete(remainder, np.where(remainder[:,1] == chem), 0)
    return out

In [249]:
toe = []

# Create ORE
ORE = Chem('ORE')
ORE.min_ore = [1,1]

for i in incom_pies:
    # Decompose entry into amount and ingredient
    amountp,ingredp = int(i[-1].split(' ')[0]), i[-1].split(' ')[1]
    if ingredp not in toe:
        vars()[ingredp] = Chem(ingredp)
    
    # Create recipie card
    recipie = []
    # first, add product to recipie
    recipie.append([amountp,vars()[ingredp]])
    
    for entry in i[:-1]:
        # Decompose entry into amount and ingredient
        amount,ingred = int(entry.split(' ')[0]), entry.split(' ')[1]
        if ingred not in toe: # add the ingredient to the toe if its not already there
            vars()[ingred] = Chem(ingred)
        # add ingredient to recipie
        recipie.append([amount,vars()[ingred]])
        
    # put recipie card into class
    vars()[ingredp].add_recipie(recipie)


In [250]:
toe[:5]

['ORE', 'GTPZ', 'DTQRC', 'RNQJN', 'NQKVW']

In [251]:
FUEL.min_ore_calc()
FUEL.min_ore

[1, 5280576]

5280576 is too high!

Crap. From what I've seen, things are working as expected, however, I wonder if I need to factor in my extra products intelligently? I've been tossing the remainders and not keeping track of them.

In [24]:
GTPZ.recipies

[[[7, <__main__.Chem at 0x5f61cc0>], [118, <__main__.Chem at 0x5f6f128>]]]

In [254]:
a = FUEL.min_ore_calc_remainders()
FUEL.min_ore

[1, 5280576]

In [255]:
a

array([[0, <__main__.Chem object at 0x00000000055E4080>],
       [0, <__main__.Chem object at 0x00000000055E49E8>],
       [0, <__main__.Chem object at 0x00000000055D59B0>],
       [0, <__main__.Chem object at 0x00000000055D5AC8>],
       [0, <__main__.Chem object at 0x00000000055FEC18>],
       [0, <__main__.Chem object at 0x00000000055CC240>],
       [0, <__main__.Chem object at 0x00000000055E4AC8>],
       [0, <__main__.Chem object at 0x00000000055E4128>],
       [0, <__main__.Chem object at 0x00000000055E4BE0>],
       [0, <__main__.Chem object at 0x00000000055E4908>],
       [0, <__main__.Chem object at 0x00000000055FED30>],
       [0, <__main__.Chem object at 0x00000000055FE4A8>],
       [0, <__main__.Chem object at 0x00000000055D58D0>],
       [0, <__main__.Chem object at 0x00000000055E47F0>],
       [0, <__main__.Chem object at 0x00000000055E4F60>],
       [0, <__main__.Chem object at 0x00000000055E4D68>],
       [0, <__main__.Chem object at 0x00000000055E4470>],
       [0, <__

In [257]:
a[-1,1].name, a[-1,1].recipies

('SPRCM',
 [[[4, <__main__.Chem at 0x55e4630>], [3, <__main__.Chem at 0x55e4d30>]]])

In [117]:
for chem in a[:,1]:
    print(np.sum([i for i in a[np.where(a[:,1] == chem),0]]))

98
96
94
42
36
134
134
128
130
128
94
30
70
70
134
134
128
130
128
94
130
128
134
134
128
94
84
32
30
134
134
128
130
128
94
70
70
30
18
70
70
134
134
128
130
128
94
130
128
134
134
128
94
84
32
30
12
6
6
4
94
98
96
94
42
36
134
134
128
130
128
94
30
94
84
98
96
94
52
24
10
10
134
134
6
94
98
96
94
42
36
134
134
128
130
128
94
30
94
84
98
96
94
52
24
10
10
4
98
96
94
42
36
134
134
128
130
128
94
30
94
84
98
96
94
52
24
8
98
96
94
42
98
96
94
42
36
134
134
128
130
128
94
30
94
84
98
96
94
52
24
6
94
84
98
96
94
52
94
84
98
96
94
52
8
8
130
128
134
134
128
94
84
32
2
2
2
98
96
94
42
36
134
134
128
130
128
94
30
94
84
98
96
94
52
24
8
98
96
94
42
36
70
70
94
84
98
96
94
52
6
4
94
84
98
96
94
52
134
134
6
4
94
84
98
96
94
52
94
84
98
96
94
52
8
8
98
96
94
42
36
134
134
128
130
128
94
30
94
84
98
96
94
52
24
8
4
70
70
134
134
128
130
128
94
130
128
134
134
128
94
84
32
30
134
134
128
130
128
94
70
70
30
18
70
70
134
134
128
130
128
94
130
128
134
134
128
94
84
32
30
12
98
96
2
2
98
96
94
42

In [105]:
sum(a[:,0])

1812

In [87]:
FUEL.recipies

[[[1, <__main__.Chem at 0x1054712e8>],
  [10, <__main__.Chem at 0x10545ec18>],
  [27, <__main__.Chem at 0x10546b9b0>],
  [9, <__main__.Chem at 0x10546bf28>],
  [41, <__main__.Chem at 0x105471470>],
  [2, <__main__.Chem at 0x1054714a8>],
  [54, <__main__.Chem at 0x10546ba58>],
  [85, <__main__.Chem at 0x10545efd0>],
  [5, <__main__.Chem at 0x105471550>],
  [2, <__main__.Chem at 0x105471588>],
  [28, <__main__.Chem at 0x10546b358>]]]

In [245]:
multiple = 3
needed = 16

math.ceil(needed/multiple)*3

6

In [63]:
multiple - (needed%multiple)

2

In [46]:
# You get 7 GTPZ from 118 core
GTPZ.min_ore 

[7, 118]

In [47]:
GTPZ.min_ore_calc()
GTPZ.min_ore

[7, 118]

In [111]:
# you get 4 VTBTK from 6 GTPZ
VTBTK.min_ore

[1, 999999999]

In [112]:
# you get 6 XCDZP for 6 VTBTK
XCDZP.min_ore

[1, 999999999]

In [224]:
np.empty((0,2)).shape

(0, 2)

In [40]:
del GTPZ

In [241]:
a=np.empty((0,2))
a = np.append(a,[[3,GTPZ]], axis=0)
a = np.append(a,[[3,VTBTK]], axis=0)
a = np.append(a,[[5,GTPZ]], axis=0)


In [244]:
DTQRC in a

False

In [150]:
a = np.delete(a, np.where(a[:,1] == VTBTK), 0)
a

array([[3, <__main__.Chem object at 0x00000000061FFD30>],
       [5, <__main__.Chem object at 0x00000000061FFD30>]], dtype=object)

In [156]:
np.where(a[:,1] == VTBTK)[0][0]

1

In [208]:
len(np.where(a[:,1] == 'boo')[0])

0

In [198]:
a = np.delete(a,np.where(a[:,1] == GTPZ),0)
a

array([[3, <__main__.Chem object at 0x0000000005528438>]], dtype=object)

In [ ]:
aaaa

In [1]:
# Just in case I need an LCM:
def gcd(a,b):
    """Compute the greatest common divisor of a and b"""
    while b > 0:
        a, b = b, a % b
    return a
    
def lcm(a, b):
    """Compute the lowest common multiple of a and b"""
    return int(a * b / gcd(a, b))

In [20]:
incom_pies[1][0]#.split(' ')[0]

'6 RNQJN'

In [36]:
a = np.empty((2))

In [37]:
a

array([-3.10503618e+231, -2.68679134e+154])

In [40]:
np.append(a,a, axis=0)

array([-3.10503618e+231, -2.68679134e+154, -3.10503618e+231,
       -2.68679134e+154])

In [41]:
a

array([-3.10503618e+231, -2.68679134e+154])

In [52]:
a = np.array([[1,1,1],[2,1,3]]).T

In [53]:
a

array([[1, 2],
       [1, 1],
       [1, 3]])